In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling2D

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_1d304de22f0c4824ad8b8f7107066adf = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_1d304de22f0c4824ad8b8f7107066adf = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_1d304de22f0c4824ad8b8f7107066adf = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='YWwiCNSSbDNko-SiDa3dLvUbseXdXZjMGIReQxZ2lbz2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_1d304de22f0c4824ad8b8f7107066adf)

streaming_body_1 = client_1d304de22f0c4824ad8b8f7107066adf.get_object(Bucket='iceburg-donotdelete-pr-1e1qh6poyfkk5q', Key='Iceberg Detection dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
pwd

'/home/wsuser/work'

In [6]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Iceberg Detection dataset/dataset/train',target_size=(75,75),batch_size= 32, class_mode='binary')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Iceberg Detection dataset/dataset/test',target_size=(75,75),batch_size= 32, class_mode='binary')

Found 1284 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [7]:
model=Sequential()

In [8]:
model.add(Conv2D(64,(3,3), activation='relu', input_shape=(75,75,3)))

In [9]:
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

In [10]:
model.add(Flatten())

In [11]:
model.add(Dense(units= 512, kernel_initializer="uniform", activation="relu"))
model.add(Dense(units = 256, kernel_initializer="uniform", activation="relu"))
model.add(Dense(units = 1, kernel_initializer="uniform", activation="sigmoid"))

In [12]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [13]:
model.fit_generator(x_train,steps_per_epoch=40, epochs= 10, validation_data=x_test, validation_steps=10)

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
40/40 [==============================] - 40s 981ms/step - loss: 0.7809 - accuracy: 0.5206 - val_loss: 0.6835 - val_accuracy: 0.5938
Epoch 2/10
40/40 [==============================] - 37s 925ms/step - loss: 0.6828 - accuracy: 0.5969 - val_loss: 0.6640 - val_accuracy: 0.7250
Epoch 3/10
40/40 [==============================] - 38s 956ms/step - loss: 0.6350 - accuracy: 0.6472 - val_loss: 0.5978 - val_accuracy: 0.7063
Epoch 4/10
40/40 [==============================] - 38s 937ms/step - loss: 0.6045 - accuracy: 0.6475 - val_loss: 0.5523 - val_accuracy: 0.7406
Epoch 5/10
40/40 [==============================] - 39s 964ms/step - loss: 0.5628 - accuracy: 0.7012 - val_loss: 0.4817 - val_accuracy: 0.7469
Epoch 6/10
40/40 [==============================] - 38s 953ms/step - loss: 0.5135 - accuracy: 0.7322 - val_loss: 0.4759 - val_accuracy: 0.7469
Epoch 7/10
40/40 [==============================] - 38s 940ms/step - loss: 0.4946 - accuracy: 0.7314 - val_loss: 0.4692 - val_accuracy: 0.7312

In [14]:
model.save("iceberg.h5")

In [15]:
import tensorflow
tensorflow.__version__


'2.4.3'

In [16]:
x_train.class_indices

{'Iceberg': 0, 'Ship': 1}

### Deployment

In [17]:
!tar -zcvf image-classification-model_new.tgz iceberg.h5

iceberg.h5


In [18]:
ls -1

'Iceberg Detection dataset'/
iceberg.h5
image-classification-model_new.tgz


In [19]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 25.3 MB/s eta 0:00:01


In [20]:
from ibm_watson_machine_learning import APIClient

In [21]:
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"iXac6viyJnOROUxEiEAIGVBGXNYm8-rsI-U4C0nI4JUy"
}

In [22]:
client = APIClient(wml_credentials=wml_credentials)

In [23]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [24]:
space_uid=guid_from_space_name(client,'iceburg_deployment')
print("Space UID=" + space_uid)

Space UID=0c4cbc44-cb18-42bd-ba83-fc3656d60b93


In [25]:
client.set.default_space(space_uid)

'SUCCESS'

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [27]:
#software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [28]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})

model_id = client.repository.get_model_uid(model_details)

In [29]:
model_id

'246fbf6b-5207-4382-a862-a982bd229bdf'

In [30]:
client.repository.download(model_id,'iceburg.tar.gz')

Successfully saved model content to file: 'iceburg.tar.gz'


'/home/wsuser/work/iceburg.tar.gz'